In [16]:
import numpy as np
import matplotlib.pyplot as plt
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
Y_hat = np.linspace(-3, 3, 100)
Y = np.random.randint(2, size=(100))

[0 1 0 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 0 0 0 1 0 1
 0 0 0 1 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 1 0
 1 0 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 0 0 0 1 1 1 0 1 1]


In [5]:
print (Y_hat.shape)
print (Y.shape)

print (Y_hat)
print (Y)

(100,)
(100,)
[-3.         -2.93939394 -2.87878788 -2.81818182 -2.75757576 -2.6969697
 -2.63636364 -2.57575758 -2.51515152 -2.45454545 -2.39393939 -2.33333333
 -2.27272727 -2.21212121 -2.15151515 -2.09090909 -2.03030303 -1.96969697
 -1.90909091 -1.84848485 -1.78787879 -1.72727273 -1.66666667 -1.60606061
 -1.54545455 -1.48484848 -1.42424242 -1.36363636 -1.3030303  -1.24242424
 -1.18181818 -1.12121212 -1.06060606 -1.         -0.93939394 -0.87878788
 -0.81818182 -0.75757576 -0.6969697  -0.63636364 -0.57575758 -0.51515152
 -0.45454545 -0.39393939 -0.33333333 -0.27272727 -0.21212121 -0.15151515
 -0.09090909 -0.03030303  0.03030303  0.09090909  0.15151515  0.21212121
  0.27272727  0.33333333  0.39393939  0.45454545  0.51515152  0.57575758
  0.63636364  0.6969697   0.75757576  0.81818182  0.87878788  0.93939394
  1.          1.06060606  1.12121212  1.18181818  1.24242424  1.3030303
  1.36363636  1.42424242  1.48484848  1.54545455  1.60606061  1.66666667
  1.72727273  1.78787879  1.84848485  1

In [13]:
TY_hat = torch.Tensor(Y_hat.reshape(-1, 1))
TY = torch.Tensor(Y.reshape(-1, 1))

In [15]:
print (TY_hat.size())
print (TY.size())

torch.Size([100, 1])
torch.Size([100, 1])


In [14]:
loss = F.nll_loss(TY, TY_hat).numpy()
plt.plot(Y_hat, loss)
plt.show()

RuntimeError: 1D target tensor expected, multi-target not supported

In [37]:
batch_size = 7
ref=np.array([[0,1,0],[0,0,1],[1,0,0]])
actions=np.array([[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]])
ip = np.random.randint(len(actions), size=(batch_size))
print (ip)
ip = torch.Tensor(ref[actions[ip]].reshape(-1, 6))
print (ip.size())
print (ip)

op = np.random.randint(len(actions), size=(batch_size))
print (op)
op = torch.Tensor(ref[actions[op]].reshape(-1, 6))
print (op.size())
print (op)

[4 5 1 6 1 2 4]
torch.Size([7, 6])
tensor([[0., 1., 0., 0., 0., 1.],
        [0., 0., 1., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 1., 0.],
        [1., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 1.]])
[0 7 6 1 5 3 7]
torch.Size([7, 6])
tensor([[1., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0.],
        [1., 0., 0., 0., 1., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 1.]])


In [21]:
loss = F.nll_loss(ip, op)
print (loss)

RuntimeError: 1D target tensor expected, multi-target not supported

In [26]:
loss = F.binary_cross_entropy(ip, op, reduction='none')
print (loss)

loss = F.binary_cross_entropy(ip, ip, reduction='none')
print (loss)

tensor([[-0.0000, -0.0000, -0.0000, 27.6310, -0.0000, 27.6310],
        [27.6310, 27.6310, -0.0000, -0.0000, 27.6310, 27.6310],
        [27.6310, -0.0000, 27.6310, 27.6310, 27.6310, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [27.6310, -0.0000, 27.6310, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, 27.6310, 27.6310, -0.0000]])
tensor([[-0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0.]])


In [35]:
batch_size = 7
ref=torch.FloatTensor([[0,1,0],[0,0,1],[1,0,0]])
actions=np.array([[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]])
ip = np.random.randint(len(actions), size=(batch_size))
print (ip)
ip = torch.Tensor(ref[actions[ip][0]].reshape(-1, 3))
print (ip.size())
print (ip)

[2 3 1 7 7 0 3]
torch.Size([2, 3])
tensor([[1., 0., 0.],
        [0., 0., 1.]])


In [27]:
u = np.random.randint(3, size=(7,3)).astype(float)
print (u)

[[0. 0. 1.]
 [2. 1. 1.]
 [1. 1. 0.]
 [1. 2. 2.]
 [2. 2. 0.]
 [1. 2. 1.]
 [2. 2. 1.]]


In [28]:
F.one_hot(torch.as_tensor(u), num_classes=3)

RuntimeError: one_hot is only applicable to index tensor.

In [2]:
u = torch.randint(3, (7, 2))
u

tensor([[1, 0],
        [2, 1],
        [2, 2],
        [2, 1],
        [2, 0],
        [2, 2],
        [1, 0]])

In [3]:
u_pred = torch.rand((7, 6))
u_pred

tensor([[0.3787, 0.5554, 0.5687, 0.2718, 0.4508, 0.5451],
        [0.0378, 0.2429, 0.7639, 0.0425, 0.3442, 0.2594],
        [0.2459, 0.5431, 0.3352, 0.7255, 0.8996, 0.3176],
        [0.0392, 0.2200, 0.8852, 0.7540, 0.9267, 0.4024],
        [0.2913, 0.0125, 0.1707, 0.2411, 0.6091, 0.6629],
        [0.9377, 0.5727, 0.8608, 0.2946, 0.2895, 0.5671],
        [0.2271, 0.6809, 0.6697, 0.8398, 0.2618, 0.9023]])

In [4]:
lx = F.cross_entropy(u_pred[:, :3], u[:, 0], reduction='none')
ly = F.cross_entropy(u_pred[:, 3:], u[:, 1], reduction='none')
print (lx)
print (ly)

tensor([1.0478, 0.7313, 1.1460, 0.6644, 1.0925, 1.0402, 0.9647])
tensor([1.2557, 0.9777, 1.4570, 0.8893, 1.3786, 0.9239, 0.9653])


In [17]:
u_pred = F.one_hot(torch.tensor(u, dtype=torch.long))
u_pred = torch.reshape(u_pred, (u_pred.size()[0], -1))
u_pred = torch.tensor(u_pred, dtype=torch.float)
u_pred = u_pred - math.pow(10, -30)
u_pred

/Users/pradeepkadubandi/.local/share/virtualenvs/DemoPlanner--5SJrBy7/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/Users/pradeepkadubandi/.local/share/virtualenvs/DemoPlanner--5SJrBy7/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


tensor([[-1.0000e-30,  1.0000e+00, -1.0000e-30,  1.0000e+00, -1.0000e-30,
         -1.0000e-30],
        [-1.0000e-30, -1.0000e-30,  1.0000e+00, -1.0000e-30,  1.0000e+00,
         -1.0000e-30],
        [-1.0000e-30, -1.0000e-30,  1.0000e+00, -1.0000e-30, -1.0000e-30,
          1.0000e+00],
        [-1.0000e-30, -1.0000e-30,  1.0000e+00, -1.0000e-30,  1.0000e+00,
         -1.0000e-30],
        [-1.0000e-30, -1.0000e-30,  1.0000e+00,  1.0000e+00, -1.0000e-30,
         -1.0000e-30],
        [-1.0000e-30, -1.0000e-30,  1.0000e+00, -1.0000e-30, -1.0000e-30,
          1.0000e+00],
        [-1.0000e-30,  1.0000e+00, -1.0000e-30,  1.0000e+00, -1.0000e-30,
         -1.0000e-30]])

In [18]:
lx = F.cross_entropy(u_pred[:, :3], u[:, 0], reduction='none')
ly = F.cross_entropy(u_pred[:, 3:], u[:, 1], reduction='none')
print (lx)
print (ly)

tensor([0.5514, 0.5514, 0.5514, 0.5514, 0.5514, 0.5514, 0.5514])
tensor([0.5514, 0.5514, 0.5514, 0.5514, 0.5514, 0.5514, 0.5514])


In [27]:
F.cross_entropy(torch.Tensor([[0,0,1]]), torch.LongTensor([2]))

tensor(0.5514)

In [48]:
u_pred = torch.rand((7, 2))
u_pred

tensor([[0.7448, 0.5683],
        [0.4792, 0.4464],
        [0.1592, 0.3000],
        [0.6917, 0.6731],
        [0.9426, 0.9293],
        [0.1921, 0.4872],
        [0.6835, 0.6908]])

In [52]:
F.binary_cross_entropy(u_pred, torch.tensor(u, dtype=torch.float))

/Users/pradeepkadubandi/.local/share/virtualenvs/DemoPlanner--5SJrBy7/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor(0.8587)

In [44]:
labels = torch.Tensor(ref[u.numpy()].reshape(u.size()[0], -1))
labels = torch.tensor(labels, dtype=torch.long).detach()
labels

/Users/pradeepkadubandi/.local/share/virtualenvs/DemoPlanner--5SJrBy7/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


tensor([[0, 0, 1, 1, 0, 0],
        [1, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 0]])

In [45]:
F.binary_cross_entropy(u_pred, labels)

RuntimeError: Expected object of scalar type Float but got scalar type Long for argument #2 'target' in call to _thnn_binary_cross_entropy_forward